# This time for sure

Alignment is modeled as masked array

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, deque
from typing import Set, List
from dataclasses import dataclass
from bitarray import bitarray
import networkx as nx
import re
import queue

import numpy as np
import numba as nb
import numpy.ma as ma

import graphviz
from IPython.display import SVG

from numba import jit

import pprint
pp = pprint.PrettyPrinter(indent=2)
debug = False

%load_ext line_profiler

In [2]:
def increase_matrix_size(current_matrix):
    """Double row count of matrix, retaining old data

    Parameter: current_matrix

    Returns: updated current_matrix with additional rows

    TODO: filter out fully masked rows (broadcast) during copying
    """
    if debug:
        print('Increasing matrix size from', current_matrix.shape)
    current_matrix_row_count, column_count = current_matrix.shape # rows, then columns
    new_matrix_row = ma.MaskedArray(
        data = [-1] * column_count,
        mask = [True] * column_count
    )
    new_matrix = ma.MaskedArray(
        data = np.append(
            current_matrix.data,
            [new_matrix_row] * current_matrix_row_count,
            0 # rows, not columns
        ),
        mask = np.append(
            current_matrix.mask,
            [new_matrix_row.mask] * current_matrix_row_count,
            0
        )
    )
    return new_matrix

## Load data

In [3]:
%run create-blocks-and-dependencies.ipynb # data will be in prioritized_blocks
# print(dependencies)

## Create witness_node (dataclass)

In [4]:
@dataclass(unsafe_hash=True)
class WitnessNode:
    """TODO: Write a docstring some day"""
    token_string: str
    witness: str
    witness_offset: int

    def __repr__(self):
        return self.token_string

# Subclass of WitnessNodeEnd so that our constructed end node will be able to identify itself
class WitnessNodeEnd(WitnessNode):
    """TODO: Write a docstring some day"""

In [5]:
# Store all witness nodes in dictionary of lists
witness_node_lists = defaultdict(list) # one per witness (keys will be sigla)
# create, for each witness:
#   nodes for real tokens
#   END node
# A start node is not created. That is on purpose.
for index, witness_siglum in enumerate(witness_sigla):
    # witness_sigla is a global set when the input data is read
    for witness_token_offset, witness_token in enumerate(witnesses[index]): # list of tokens in a single witness
        witness_node_lists[witness_siglum].append(WitnessNode(witness_token, witness_siglum, witness_token_offset))
    witness_node_lists[witness_siglum].append(WitnessNodeEnd('END', witness_siglum, len(token_array)))

## Create candidate vectors from block

In [6]:
def create_candidate_vectors(block):
    """Returns list of data for block, one numpy array per hyperedge"""
    candidate_vectors = [] # list of individual vectors, not a matrix
    for token_offset_in_block in range(block.token_count):
        # first add the token_offset_in_block to the block start positions
        offset_start_positions = [value + token_offset_in_block for value in block.all_start_positions]
        tokens_to_place = [ # list of all token positions, not just first in each witness
            (
                token_membership_array[token_array_index],
                token_witness_offset_array[token_array_index]
            )
            for token_array_index in offset_start_positions
        ]

        data_for_new_vector = [0] * len(witness_sigla) # initialize to meaningless values
        mask_for_new_vector = [True] * len(witness_sigla) # we'll unmask individual values as needed
        for witness_number, witness_offset in tokens_to_place:
            data_for_new_vector[witness_number] = witness_offset
            mask_for_new_vector[witness_number] = False
        candidate_vectors.append(ma.MaskedArray(data=data_for_new_vector, mask=mask_for_new_vector))

    return candidate_vectors

## Place vectors if allowed

In [7]:
def check_whether_okay_to_place(current_vectors, potential_vector) -> bool:
    """Return True iff we can add row without creating transpositions

    current_vectors: vector space before new addition
    potential_vector: we check whether this can be added

    If subtracting a potential vector from any existing vector would return
    values that diverge in sign, the potential would cross the existing one

    If it's okay to place, we need to call merge_vector() to see whether we
    should merge"""
    subtractionResult = current_vectors - potential_vector
    signs = np.sign(subtractionResult)
    okayToPlace = (signs.min(axis=1) == signs.max(axis=1)).all()
    return True if okayToPlace is ma.masked else okayToPlace

In [8]:
@nb.njit
def merge_vectors(existing_data, existing_mask, new_data, new_mask):
    result_data = np.maximum(existing_data, new_data)
#     length = existing_data.shape[0]
#     result_data = np.array([-1] * length) # initialize vector of -1 values
#     for pos in nb.prange(length):
#         result_data[pos] = max(result_data[pos], new_data[pos])
    result_mask = existing_mask * new_mask
    return result_data, result_mask

In [9]:
def add_new_vector(input_tuple, new_vector):
    """Add row to matrix and update pointer to next empty row

    Parameters:
        input_tuple : current_matrix, pointer to next empty row
        new_vector : masked array vector to add as new row

    Returns tuple of:
        updated matrix, updated pointer
    """
    current_matrix, pointer = input_tuple
    current_matrix_row_count = current_matrix.shape[0]
    if pointer == current_matrix_row_count: # need more rows now!!!
        current_matrix = increase_matrix_size(current_matrix)
    current_matrix[pointer] = new_vector
    pointer += 1
    return (current_matrix, pointer)

In [10]:
@nb.njit(parallel=False)
def create_filter(current_matrix_data, current_matrix_mask, candidate_data, candidate_mask):
    row_count = current_matrix_data.shape[0]
    column_count = current_matrix_data.shape[1]
    filter = np.array([False] * row_count ) # will hold vector of booleans, True if row contains unmasked zero value
    for row_number in nb.prange(row_count):
        prod = 1
        for cell in range(column_count):
#             if prod == 0:
#                 break
            operand = 1 if current_matrix_mask[row_number][cell] or candidate_mask[cell] else current_matrix_data[row_number][cell] - candidate_data[cell]
            prod *= operand
        if prod == 0:
            filter[row_number] = True
    return filter

# for each row:
#   for each cell:
#     if either current matrix mask == True or candidate mask == True:
#       operand = 1
#     else:
#       operand = current data - candidate data
#
# Elvis: outcome_1 if condition else outcome_2

In [11]:
t_current_matrix = ma.MaskedArray(
    data= [[1, 2, 3, 4],
          [-1, -1, -1, -1],
          [1, -1, -1, -1],
          [3,5,6,4]],
    mask=[[False, False, False, False],
         [True, True, True, True],
         [False, True, True, True],
         [False, False, False, False]],
    fill_value = -1
)
t_candidate = ma.MaskedArray(
    data=[1, 2,- 1, -1],
    mask=[False, False, True, True],
    fill_value = -1
)
result = create_filter(t_current_matrix.data, t_current_matrix.mask, t_candidate.data, t_candidate.mask)
print(f"{result=}")

# This is done. Does it work?
#
# Rewritten create_filter() function should return vector of booleans:
# True iff the product of the row is 0
# False if the product of the row is not 0
# False if the entire row is masked

result=array([ True, False,  True, False])


In [12]:
@nb.njit(parallel=True, fastmath=True)
def find_max_values_per_column(current_matrix_data, current_matrix_mask, filter):
    # current_matrix is existing matrix (not yet looking at candidate)
    # filter is vector of booleans
    # return two vectors, values, and mask
    row_count, column_count = current_matrix_data.shape
    max_values_data = np.array([-1] * column_count) # initialize result to all masked values
    max_values_mask = np.array([True] * column_count)
    for row_number in nb.prange(row_count):
        if filter[row_number]: # process row only if filter value is True
            # TODO: try without this initial filter, too, and profile
            for cell_number in nb.prange(column_count): # each cell in current row
                # if call is masked, do nothing
                # if there already a data value for a column, do nothing
                # otherwise write
                if current_matrix_mask[row_number][cell_number] == False and \
                max_values_mask[cell_number]: # if mask is true, we haven't set a value yet
                    max_values_data[cell_number] = current_matrix_data[row_number][cell_number]
                    max_values_mask[cell_number] = False
    return max_values_data, max_values_mask

In [13]:
def add_or_merge_new_vector_into_matrix(input_tuple, candidate):
    """Return max row values (copy) and indices of rows to update in existing matrix

    Parameters:
        input_tuple : (existing matrix, pointer to next empty row in matrix)
        candidate: new vector

    Returns tuple of:
        merged_vector : vector to replace first row to update
        indices : vector of offsets of rows to update (first) or mask (others)

    filter contains vector of booleans, with True for rows in current that are merge candidates
    """

    current_matrix, pointer = input_tuple
    # Use arithmetic instead of comparison where possible
    filter = create_filter(current_matrix.data, current_matrix.mask, candidate.data, candidate.mask)
    indices = np.where(filter == True)[0] # row numbers where boolean is True
    if indices.size == 0: # if indices is empty, add new row, update current_matrix and pointer
        current_matrix, pointer = add_new_vector(input_tuple, candidate)
    #
    ###
    # RESUME HERE
    #
    # Following line is hottest because of number of hits
    # Fix by reducing the number of blocks that make it this far
    ###
    #
    elif indices.size == 1:
        # compiling the comparison separately is slower than doing it here
        if np.array_equal(current_matrix[indices[0]].mask, candidate.mask): # arrays are the same
            pass
        elif np.array_equal( # candidate is subset of current
            # current:   1 1 - (F F T)
            # candidate: 1 - - (F T T)
            # or result:       (F T T)
                (current_matrix[indices[0]].mask | candidate.mask),
                candidate.mask
            ):
                pass
        elif np.array_equal( # candidate has something new to add
            (current_matrix[indices[0]].mask | candidate.mask),
            current_matrix[indices[0]].mask
        ):
            current_matrix[indices[0]] = candidate
        else: # worst case, neither is a subset, do a full (expensive) merge
            merged_vector_data, merged_vector_mask = merge_vectors(current_matrix[indices[0]].data, current_matrix[indices[0]], candidate.data, candidate.mask)
            new_row = ma.MaskedArray(data = merged_vector_data, mask = merged_vector_mask)
            current_matrix[indices[0]] = new_row # replace first row to replace with merge
    else: # multiple merge candidates
        # First merge row is subset of candidate: merge (fairly rare)
        # If the filter length > 1, always merge (rare; reexamine later if necessary)
        max_vector_data, max_vector_mask = find_max_values_per_column(current_matrix.data, current_matrix.mask, filter)
        merged_vector_data, merged_vector_mask = merge_vectors(max_vector_data, max_vector_mask, candidate.data, candidate.mask)

        new_row = ma.MaskedArray(data = merged_vector_data, mask = merged_vector_mask)
        rows_to_change = indices
        current_matrix[rows_to_change[0]] = new_row # replace first row to replace with merge
        column_count = current_matrix.shape[1] # get column count
        current_matrix[rows_to_change[1:]] = ma.MaskedArray( # mask other rows to replace
            data=[-1] * column_count,
            mask=[True] * column_count
        )
    return (current_matrix, pointer)

## Process blocks

In [14]:
def process_blocks(input_tuple, selected_blocks):
    #input_tuple is (current_matrix, pointer) (pointer is the next available row in the matrix)
    #selected_blocks is a list of tuples, each of which is a Block and an integer, representing its dependency position
#     print("Now processing block:")
#     pp.pprint(selected_blocks)
    dependency_bitarray = bitarray(len(selected_blocks)) # track dependencies
    dependency_bitarray.setall(False) # true = no need to place; initially all false
    for current_block, dependency_pointer in selected_blocks:
        if dependency_bitarray[dependency_pointer]: # block (or superset) already placed, so skip
#             print('Skipping dependency position', dependency_pointer, ':', current_block)
            continue
        else:
            dependency_bitarray[dependency_pointer] = True # no need to revisit this block
            # Create vectors for entire block
            new_vectors = create_candidate_vectors(current_block)
            # Check only first and last, merge is both are okay
            # NB: Checking only first and last to test entire block will break with discontinuous blocks
            #   (should we ever switch to working with discontinuous blocks)
            if input_tuple[1] == 0: # no need to check for transpositions the first time; just place
                for potential_vector in new_vectors:
                    input_tuple = add_new_vector(input_tuple, potential_vector)
                while dependency_pointer in dependencies: # TODO: alternative to updating variable ("not very beautiful")
#                     print(f"Found a dependency of {dependency_pointer} to {dependencies[dependency_pointer]}")
                    dependency_bitarray[dependencies[dependency_pointer]] = True
                    dependency_pointer = dependencies[dependency_pointer]
            else:
                no_transposition = \
                    check_whether_okay_to_place(input_tuple[0], new_vectors[0]) and \
                    check_whether_okay_to_place(input_tuple[0], new_vectors[-1])
                if no_transposition: # all-or-nothing
                    for potential_vector in new_vectors:
                        input_tuple = add_or_merge_new_vector_into_matrix(input_tuple, potential_vector)
                    while dependency_pointer in dependencies: # TODO: alternative to updating variable ("not very beautiful")
#                         print(f"Found a dependency of {dependency_pointer} to {dependencies[dependency_pointer]}")
                        dependency_bitarray[dependencies[dependency_pointer]] = True
                        dependency_pointer = dependencies[dependency_pointer]
                else:
                    pass
    return input_tuple[0] # return updated matrix; TODO: do we need to return pointer, too?

In [15]:
# def priority(block: Block) -> float:
#     '''Priority ranges from 0 to ∞

#     depth (number of witnesses) / (frequency * length)
#         modified (by trial and error) to weight the components
#     scale: # TODO: how can we set these in a generally meaningful way?
#         high depth (more witnesses) is most important
#         low frequency (less repetition) is next most important
#         high length (token count) is least important
#     higher numbers are better
#         distance between neighboring values is irrelevant; all that matters is order
#     '''
#     # score = pow(block.witness_count,4) / (pow(block.frequency,3) * block.token_count)
#     score = pow(block.witness_count,6)  * block.token_count / pow(block.frequency,3)
#     return score

In [16]:
# def sort_blocks_by_priority (_blocks: List[Block]) -> List[tuple]:
#     """Return sorted tuples of (priority, pointer-to-block-in-dependency-order)"""
# #     return sorted(_blocks, key=lambda x: priority(x), reverse=True)
#     results = []
#     for index, block in enumerate(_blocks):
#         block_priority = priority(block)
#         results.append((block_priority, index))
#     return sorted(results, reverse=True)

## Main

In [17]:
# create all vectors (gingerly)
# We exclude blocks with repetition (temporarily?)
# selected_blocks = filter(lambda x: len(x.all_start_positions) == x.witness_count, prioritized_blocks)
# selected_blocks = [(blocks[position], position) for priority, position in prioritized_blocks]
selected_blocks = prioritized_blocks

# set up matrix for vectors
# column count is number of witnesses
# initial row count is equal to length of longest witness
witness_count = len(witness_node_lists)
max_witness_length = max([len(witness_node_lists[w]) for w in witness_node_lists])
alignment_matrix = ma.MaskedArray(
    data = [ma.MaskedArray(
        data = [-1] * witness_count,
        mask = [True] * witness_count
    )] * max_witness_length
)
pointer = 0

%lprun -f add_or_merge_new_vector_into_matrix process_blocks((alignment_matrix, pointer), selected_blocks)
# alignment_matrix = process_blocks((alignment_matrix, pointer), selected_blocks)

Timer unit: 1e-06 s

Total time: 31.2217 s

Could not find file /tmp/ipykernel_850/2022358424.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2                                           
     3                                           
     4                                           
     5                                           
     6                                           
     7                                           
     8                                           
     9                                           
    10                                           
    11                                           
    12                                           
    13                                           
    14                                         

In [35]:
# List of "None" values is an artifact of the print() statement
[print(index,vector) for index,vector in enumerate(alignment_matrix)]

0 [178 177 177 180]
1 [179 178 178 181]
2 [180 179 179 182]
3 [181 180 180 183]
4 [182 181 181 184]
5 [183 182 182 185]
6 [184 183 183 186]
7 [185 184 184 187]
8 [186 185 185 188]
9 [187 186 186 189]
10 [188 187 187 190]
11 [189 188 188 191]
12 [190 189 189 192]
13 [191 190 190 193]
14 [192 191 191 194]
15 [193 192 192 195]
16 [194 193 193 196]
17 [195 194 194 197]
18 [196 195 195 198]
19 [197 196 196 199]
20 [198 197 197 200]
21 [199 198 198 201]
22 [200 199 199 202]
23 [201 200 200 203]
24 [202 201 201 204]
25 [203 202 202 205]
26 [204 203 203 206]
27 [205 204 204 207]
28 [206 205 205 208]
29 [207 206 206 209]
30 [208 207 207 210]
31 [209 208 208 211]
32 [210 209 209 212]
33 [211 210 210 213]
34 [212 211 211 214]
35 [213 212 212 215]
36 [214 213 213 216]
37 [215 214 214 217]
38 [216 215 215 218]
39 [217 216 216 219]
40 [218 217 217 220]
41 [219 218 218 221]
42 [220 219 219 222]
43 [221 220 220 223]
44 [222 221 221 224]
45 [223 222 222 225]
46 [224 223 223 226]
47 [225 224 224 227]
48


524 [465 464 464 467]
525 [466 465 465 468]
526 [467 466 466 469]
527 [127 126 126 129]
528 [128 127 127 130]
529 [129 128 128 131]
530 [130 129 129 132]
531 [131 130 130 133]
532 [132 131 131 134]
533 [133 132 132 135]
534 [134 133 133 136]
535 [135 134 134 137]
536 [136 135 135 138]
537 [137 136 136 139]
538 [138 137 137 140]
539 [139 138 138 141]
540 [140 139 139 142]
541 [141 140 140 143]
542 [142 141 141 144]
543 [143 142 142 145]
544 [144 143 143 146]
545 [145 144 144 147]
546 [146 145 145 148]
547 [147 146 146 149]
548 [148 147 147 150]
549 [149 148 148 151]
550 [150 149 149 152]
551 [151 150 150 153]
552 [152 151 151 154]
553 [153 152 152 155]
554 [154 153 153 156]
555 [155 154 154 157]
556 [156 155 155 158]
557 [157 156 156 159]
558 [158 157 157 160]
559 [159 158 158 161]
560 [160 159 159 162]
561 [161 160 160 163]
562 [162 161 161 164]
563 [163 162 162 165]
564 [164 163 163 166]
565 [165 164 164 167]
566 [166 165 165 168]
567 [167 166 166 169]
568 [168 167 167 170]
569 [169 

839 [115 114 114 117]
840 [116 115 115 118]
841 [117 116 116 119]
842 [118 117 117 120]
843 [119 118 118 121]
844 [120 119 119 122]
845 [121 120 120 123]
846 [122 121 121 124]
847 [123 122 122 125]
848 [124 123 123 126]
849 [125 124 124 127]
850 [-- 125 125 128]
851 [-- -- 355 358]
852 [-- -- 555 558]
853 [-- -- 556 559]
854 [-- -- 557 560]
855 [-- -- 558 561]
856 [-- -- 559 562]
857 [-- 556 560 563]
858 [-- 678 682 685]
859 [356 355 -- --]
860 [557 555 -- --]
861 [-- -- -- --]
862 [-- -- -- --]
863 [-- -- -- --]
864 [-- -- -- --]
865 [-- -- -- --]
866 [-- -- -- --]
867 [-- -- -- --]
868 [-- -- -- --]
869 [-- -- -- --]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [33]:
print(len(alignment_matrix))

870


## Create variant graph from numpy masked array

In [19]:
class VG_node():
    """Variant graph node"""
    def __init__(self, token_string, data): # dictionary of siglum:witness_offset
        self.token_string = token_string
        self.__dict__.update(**data)
        self._sigla = [key for key in data.keys()]
    def __repr__(self):
        return self.token_string + "~" + "|".join([":".join([str(key), str(getattr(self, key))]) for key in self.sigla()])
    def __setitem__(self, key, value):
        self._sigla.append(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def sigla(self):
        return self._sigla
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [20]:
# Create networkx variant graph from our (not networkx) alignment graph
# Create variant graph edges for start and end nodes (only)
# We've treated VG as a global and we refer to it in the earlier functions; should we pass it instead?
def augment_or_add_edge_without_conversion(siglum, source_VG_node, target_VG_node):
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
        )

In [21]:
# Use alignment graph node to look up variant graph node
def alignment_node_to_VG_node(alignment_node: WitnessNode) -> VG_node:
    global witness_offset_to_VG_node
    return witness_offset_to_VG_node[alignment_node.witness][alignment_node.witness_offset]

In [22]:
# Create variant graph data edges (except start and end nodes)
def augment_or_add_edge(siglum, source, target):
    source_VG_node = alignment_node_to_VG_node(source)
    target_VG_node = alignment_node_to_VG_node(target)
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
                )

In [23]:
def create_variant_graph_from_vector_space(alignment_matrix):
    # create variant graph and add start and end nodes

    start_offsets = {} # make sure that START points to first tokens of each witness
    for siglum in witness_sigla:
        start_offsets[siglum] = -1;
    start_node = VG_node("START", start_offsets)
    end_node = VG_node("END", {})
    global VG # does this have to be global; we return it at the end, but we also use it in functions above
    VG = nx.DiGraph(start = start_node, end = end_node) # create start and end properties to find terminal nodes
    VG.add_node(start_node)
    VG.add_node(end_node)

    # keep track of which witness nodes (and therefore also hyperedges)
    # have been added to VG
    # does this duplicate information available from the following structure, since
    #   if we assign a value below to replace the None, this doesn't seem to add anything
    #   Can we get rid of this and use witness_offset_to_VG_node for this purpose instead?
    # NB: Includes END (not START), which we will later ignore (we think …)
    from bitarray import bitarray
    global VG_tracking # keep this, but should we return it instead of making it global?
    VG_tracking = {}
    for siglum, witness_node_list in witness_node_lists.items():
        VG_tracking[siglum] = bitarray(len(witness_node_list))
        VG_tracking[siglum].setall(0)

    # map from witness token node to variant graph node (needed to construct edges)
    # values will be added as we create variant graph nodes
    # NB: Includes END (not START), which we will later ignore (we think …)
    global witness_offset_to_VG_node # does this have to be global?
    witness_offset_to_VG_node = {}
    for siglum in witness_node_lists.keys():
        witness_offset_to_VG_node[siglum] = [None] * (len(witness_node_lists[siglum]))

    # Replaces old code in following cell to build variant graph,
    #   this time using vector space as main data source

    # 1. Convert vectors to variant graph nodes

    for row in alignment_matrix:
        if debug:
            print(' '.join(('Processing', str(row))))
        if row.all() is ma.masked: # don't process fully masked rows
            continue
        else:
            # create dictionary of siglum:value for node (variable name: data) and update globals
            data = {}
            for index, value in enumerate(row):
                if value != alignment_matrix.fill_value:
                    siglum = sigla[index]
                    value = int(value)
                    data[siglum] = value # add to eventually data for new VG node
                    VG_tracking[siglum][value] = 1 # update global; do we need to adjust by 1?
            # get token string for node (variable name: token_string)
            siglum, offset = next(iter(data.items()))
            token_string = witness_node_lists[siglum][int(offset)].token_string
            # create and add new VG_node
            new_VG_node = VG_node(token_string, data)
            VG.add_node(new_VG_node)
            for siglum, offset in data.items(): # Eek! Another for loop! How embarrassing!
                witness_offset_to_VG_node[siglum][offset] = new_VG_node # update other global;

    # 2. Traverse variant graph (in arbitrary order), each of which contains witness nodes.
    #    Draw outgoing edges, which point to nodes with next tokens in each witness present
    #      on the variant graph node.
    #    Creating new variant graph nodes for witness tokens not in a alignment vector.
    #    Use queue because we're iterating over a dynamic structure (inventory of variant graph nodes).

    VG_node_queue = queue.Queue()
    #
    # Temporarily skipping START and END, which are the first two nodes
    #
    for node in VG.nodes(): # add all initial VG nodes to queue except START and END
        VG_node_queue.put(node)
        if debug:
            print(f"Adding node to VG_node_queue; length is {VG_node_queue.qsize()=}")
#     ignore_start = VG_node_queue.get() # temporarily ignore START node when creating edges
#     ignore_end = VG_node_queue.get() # temporarily ignore END node when creating edges
#     if debug:
#         print(f"Removed START and END; length is {VG_node_queue.qsize()=}")
    while not VG_node_queue.empty():
        if debug:
            print(f"Processing source node from VG_node_queue; length is {VG_node_queue.qsize()=}")
        source_node = VG_node_queue.get()
        targets = set()
        edge_labels = defaultdict(list) # key is target VG node, value is list of sigla
        source_sigla = source_node._sigla # all sigla on source node
        for siglum in source_sigla:
            # target may or may not already exist as node in VG
            source_offset = source_node[siglum]
            target_offset = source_offset + 1
            target = witness_offset_to_VG_node[siglum][target_offset]
            if target: # does the target already exist?:
                targets.add(target)
                edge_labels[target].append(siglum)
            else: # 1) create target, 2) add to targets, 3) add to queue of nodes, and 4 + 5) update globals
                # TODO: Do we need to update globals? We shouldn't need to return to nodes we add here.
                # NB: If real target object is of type WitnessNodeEnd, don't add it to the queue or create an edge
                # print(f"{type(witness_node_lists[siglum][target_offset])=}")
                witness_node_target = witness_node_lists[siglum][target_offset]
                if type(witness_node_target) != WitnessNodeEnd:
                    target_token_string = witness_node_target.token_string
                    new_VG_node = VG_node(target_token_string, {siglum: target_offset}) # 1 create target
                    targets.add(new_VG_node) # 2 add to targets
                    edge_labels[new_VG_node].append(siglum)
                    VG_node_queue.put(new_VG_node) # 3 add to queue of nodes
                    VG_tracking[siglum][target_offset] = 1 # 4 update first global
                    witness_offset_to_VG_node[siglum][target_offset] = new_VG_node # 5. update second global
        for target in targets:
            VG.add_edge(source_node, target, label=",".join(edge_labels[target]))
    #
    # Add edges for end VG node
    #
    all_end_data_nodes = defaultdict(list)
    for siglum in witness_sigla:
        # Figure out why -2 works and clean up as needed
        key = witness_offset_to_VG_node[siglum][-2]
        all_end_data_nodes[key].append(siglum)
    for key,value in all_end_data_nodes.items():
        VG.add_edge(key, end_node, label=",".join(value))

    # we're done! return the result
    return VG

In [24]:
# create output
VG = create_variant_graph_from_vector_space(alignment_matrix)

In [25]:
# alignment_matrix.shape

## Visualize variant graph

In [26]:
# # node id values must be strings for graphviz
# a = graphviz.Digraph(format="svg", name="variant_graph_unjoined")
# a.attr(rankdir = "LR")
# a.attr(rank = 'same')
# a.attr(compound='true')

# # plot nodes, building {node: id} for lookup
# node_to_id = {}
# for index, node in enumerate(VG.nodes()):
#     node_id = str(index)
#     node_to_id[node] = node_id
#     if node is None:
#         node_text = 'NONE'
#     else:
#         node_text = node.token_string + " (" + node_id + ")"
#     a.node(node_id, label=node_text)

# # plot edges
# for edge in VG.edges(data=True):
#     # edge is a three-item tuple: source, target, dictionary of properties
#     a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=edge[2]["label"])

# print('aligning', how_many_paragraphs, 'paragraphs') # confirm
# SVG(a.view())

In [27]:
#  This function joins the variant graph in place.
#  This function is a straight port of the Java version of CollateX.

def join(graph):
    processed = set()
    end = graph.graph["end"]
    queue = deque()
    for (_, neighbor) in graph.out_edges(graph.graph["start"]):
        queue.appendleft(neighbor)
    while queue:
        vertex = queue.popleft()
        out_edges = graph.out_edges(vertex)
        if len(out_edges) == 1:
            (_, join_candidate) = next(iter(out_edges))
            can_join = join_candidate != end and len(graph.in_edges(join_candidate)) == 1
            if can_join:
                join_vertex_and_join_candidate(graph, join_candidate, vertex)
                # we have merged join_candidate (tokens originally to the right) into vertex (tokens originally to the left)
                # (for now, join_candidate node and all of its edges are still there, and we need to remove edges before
                #   we can remove node, which we do in a for loop)
                #
                # RESUME HERE: both of the following branches are wrong, the first cryptically (it shows sigla, but not
                #   all sigla) and the second conspicuously
                #
                for (_, neighbor, data) in list(graph.out_edges(join_candidate, data=True)):
                    graph.remove_edge(join_candidate, neighbor)
                    if graph.has_edge(vertex, neighbor):
                        continue # TODO: this looks wrong
#                         graph[source][target]["siglum"].append(data["siglum"])
                    else:
#                         graph.add_edge(vertex, neighbor, siglum=data["siglum"])
                         graph.add_edge(vertex, neighbor,label=data["label"])
                graph.remove_edge(vertex, join_candidate)
                graph.remove_node(join_candidate)
                queue.appendleft(vertex)
                continue
        processed.add(vertex)
        for (_, neighbor) in out_edges:
            # FIXME: Why do we run out of memory in some cases here, if this is not checked?
            if neighbor not in processed:
                queue.appendleft(neighbor)


def join_vertex_and_join_candidate(graph, join_candidate, vertex):
    # Note: since there is no normalized/non normalized content in the graph
    # a space character is added here for non punctuation tokens

    if re.match(r'^\W', join_candidate.token_string):
        vertex.token_string += join_candidate.token_string
    else:
        vertex.token_string += (" " + join_candidate.token_string)
    # join_candidate must have exactly one token (inside a list); left item may have more
#     for siglum, token in join_candidate.tokens.items():
#         vertex.add_token(siglum, token[0])

In [28]:
# len(nx.algorithms.cycles.find_cycle(VG))

In [29]:
# nx.algorithms.cycles.find_cycle(VG)

In [30]:
# We need this to create a joined variant graph
# Commented out temporarily
#
join(VG)

# node id values must be strings for graphviz
a = graphviz.Digraph(format="svg", name="variant_graph_joined")
a.attr(rankdir = "LR")
a.attr(rank = 'same')
a.attr(compound='true')

# plot nodes, building {node: id} for lookup
node_to_id = {}
for index, node in enumerate(VG.nodes()):
    node_id = str(index)
    node_to_id[node] = node_id
    if node is None:
        node_text = 'None'
    else:
        node_text = node.token_string + " (" + node_id + ")"
    a.node(node_id, label=node_text)

# plot edges
for edge in VG.edges(data=True):
    # edge is a three-item tuple: source, target, dictionary of properties
#     label = "(all)" if len(edge[2]["siglum"]) == len(witness_sigla) else ",".join(sorted(edge[2]["siglum"]))
    label = edge[2]["label"]
    a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=label)

print('aligning', how_many_paragraphs, 'paragraphs') # confirm
SVG(a.view())

aligning 3 paragraphs


In [31]:
# Find first non-masked value in each column to avoid traversing entire column
# In our matrix, all non-masked column values for rows that are merge candidates
#   have the same value. Previously we found max(), which had to look at them all
#   and compare, but the first value will necessarily also be the max().

# current_matrix = ma.MaskedArray(
#     data =[[-1, 2, 3], [4, -1,6], [7,8,-1]],
#     mask=[[True, False, False],[False, True, False],[False, False, True]],
#     fill_value=-1
# )
# columns = current_matrix.T # swap rows and columns and process (new) rows
# [print(column) for column in columns]
# first_non_masked = [column[column.mask == False][0] for column in columns]
# first_non_masked

In [32]:
VG.nodes

NodeView((START~w0:-1|w1:-1|w2:-1|w3:-1, END~, when the organisation has once begun to vary, it generally continues to vary for many generations. No case is on record of a variable being ceasing to be variable under cultivation. Our oldest cultivated plants, such as wheat, still often yield new varieties: our oldest domesticated animals are still capable of rapid improvement or modification. It has been disputed at what period of life the causes of variability, whatever they may be, generally act; whether during the early or late period of development of the embryo, or at the instant of conception. Geoffroy St. Hilaire's experiments show that unnatural treatment of the embryo causes monstrosities; and monstrosities cannot be separated by any clear line of distinction from mere variations. But I am strongly inclined to suspect that the most frequent cause of variability may be attributed to the male and female reproductive elements having been affected prior to the act of conception. Se

In [39]:
a = np.array([127, 128, 129, 0], dtype=np.int16)

In [40]:
a

array([127, 128, 129,   0], dtype=int16)